In [1]:
import numpy as np


In [31]:
class TicTacToe:
  def __init__(self) -> None:
    self.column_size=3
    self.row_size=3
    self.action_size=self.column_size*self.row_size
  def get_initial_state(self):
    return np.zeros((self.column_size,self.row_size))
  def get_next_state(self,state,action,player):
    row=action//self.column_size;
    column=action % self.column_size;
    state[row,column]=player;
    return state;
  def get_valid_state(self,state):
    return (state==0).reshape(-1).astype(np.uint8)
  def check_win(self,state,action):
    row=action//self.column_size;
    col=action%self.column_size
    diagnol_sum=np.diag(state).sum()
    anti_diagnol_sum=np.diag(np.fliplr(state)).sum()
    row_sum=state[row,:].sum()
    col_sum=state[:,col].sum()
    player=state[row,col]
    print(self.column_size,player)
    if diagnol_sum==player*self.column_size or anti_diagnol_sum==player*self.column_size:
      return True
    if row_sum==player*self.column_size or col_sum==player*self.column_size:
      return True
    return False
  def get_value_terminated(self,state,action):
    result=self.check_win(state,action)
    if result ==True:
      return 1, True
    if np.sum(self.get_valid_state(state))==0:
      return 0,True
    return 0,False

In [3]:
 state=np.zeros((4,4))
 (state==0).reshape(-1).astype(np.uint8)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [36]:
game=TicTacToe()
state=game.get_initial_state()
player=1;
for i in range(10):
  actions=game.get_valid_state(state)
  action=np.random.choice(np.where(actions==1)[0])
  state=game.get_next_state(state,action,player)
  print(state)
  reward,termination=game.get_value_terminated(state,action)
  if termination==True:
    if reward==1:
     print("player ",player,"won")
    else:
     print("game resulted in a draw")
    break;
  player=-player;


[[0. 0. 0.]
 [0. 0. 0.]
 [1. 0. 0.]]
3 1.0
[[ 0.  0.  0.]
 [-1.  0.  0.]
 [ 1.  0.  0.]]
3 -1.0
[[ 0.  0.  0.]
 [-1.  0.  1.]
 [ 1.  0.  0.]]
3 1.0
[[ 0.  0.  0.]
 [-1. -1.  1.]
 [ 1.  0.  0.]]
3 -1.0
[[ 1.  0.  0.]
 [-1. -1.  1.]
 [ 1.  0.  0.]]
3 1.0
[[ 1.  0.  0.]
 [-1. -1.  1.]
 [ 1.  0. -1.]]
3 -1.0
[[ 1.  1.  0.]
 [-1. -1.  1.]
 [ 1.  0. -1.]]
3 1.0
[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1.  0. -1.]]
3 -1.0
[[ 1.  1. -1.]
 [-1. -1.  1.]
 [ 1.  1. -1.]]
3 1.0
game resulted in a draw


In [23]:
game=TicTacToe()
state=game.get_initial_state()
actions=game.get_valid_state(state)
action=np.random.choice(np.where(actions==1)[0])
game.check_win(state,action)

3 0.0


True

In [5]:
game1=TicTacToe()
state=game1.get_initial_state()
actions=game1.get_valid_state(state)
np.random.choice(np.where(actions==1)[0])
# np.random(np.choice(np.where(actions==1))

np.int64(7)

In [40]:
!git config --global user.email "prakharvision@gmail.com"
!git config --global user.name "prakhar"
!git commit -m "created monte carlo search tree program and which runs a simulation based on the random choices taken"


[master (root-commit) b6d3ad4] created monte carlo search tree program and which runs a simulation based on the random choices taken
 21 files changed, 51025 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.04.30/13.36.29.848398.log
 create mode 100644 .config/logs/2025.04.30/13.36.50.566118.log
 create mode 100644 .config/logs/2025.04.30/13.36.58.979052.log
 create mode 100644 .config/logs/2025.04.30/13.37.00.207659.log
 create mode 100644 .config/logs/2025.04.30/13.37.08.828246.log
 create mode 100644 .config/logs/202

In [41]:
!git remote add origin https://github.com/pacificDeveloperPrakhar/TicTacToe_play_using_mcts.git
!git branch -M main
!git push -u origin main

fatal: could not read Username for 'https://github.com': No such device or address
